# Laboratory excercise 5

The notebook contains excercise connected to the auditory excerice 6. For any questions feel free to contact assistant: eda.jovicic@fer.hr

In [52]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import pathlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
%matplotlib inline

# set seed for consistency
np.random.seed(2)

1. Load the cleaned dataset from the first exercise. If you haven't saved the dataset, rerun the exercise and save the final dataset.

In [16]:
#load data
org_df = pd.read_csv(f"{str(pathlib.Path().resolve())[:-1]}1\\lab1.csv", index_col=0)
df = org_df.copy()
display(df)

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,2,1,standard,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0
1,female,2,5,standard,1,1,2,1,0.0,2,0,69,90,88,1.0,0.0,0.0,1.0
2,female,1,3,standard,1,2,2,1,4.0,1,1,87,93,91,1.0,0.0,0.0,1.0
3,male,0,0,free/reduced,1,1,0,0,1.0,2,0,45,56,42,0.0,1.0,1.0,0.0
4,male,2,5,standard,1,1,2,1,0.0,1,0,76,78,75,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,female,3,2,standard,1,2,2,0,2.0,1,0,59,61,65,1.0,0.0,0.0,1.0
29113,male,4,2,standard,1,2,1,0,1.0,0,0,58,53,51,0.0,1.0,0.0,1.0
29114,female,2,2,free/reduced,0,1,2,0,1.0,0,0,61,70,67,1.0,0.0,1.0,0.0
29115,female,3,0,standard,0,1,1,0,3.0,1,0,82,90,93,1.0,0.0,0.0,1.0


2. Using linear regression, model the MathScore using features: TestPrep, NrSiblings, ParentMaritalStatus and ReadingScore. 

In [17]:
# Declares the model

y = df.MathScore
iksevi = ["TestPrep", "NrSiblings", "ParentMaritalStatus", "ReadingScore"]
X = df[iksevi]

X.describe()

,TestPrep,NrSiblings,ParentMaritalStatus,ReadingScore
count,29117.000000,29117.000000,29117.000000,29117.000000
mean,0.675276,2.147714,1.109077,69.390081
std,0.468280,1.428635,0.676667,14.760190
min,0.000000,0.000000,0.000000,10.000000
25%,0.000000,1.000000,1.000000,59.000000
50%,1.000000,2.000000,1.000000,70.000000
75%,1.000000,3.000000,2.000000,80.000000
max,1.000000,26.000000,3.000000,100.000000


In [18]:
y.describe()

count    29117.000000
mean        66.559948
std         15.344253
min          0.000000
25%         56.000000
50%         67.000000
75%         78.000000
max        100.000000
Name: MathScore, dtype: float64

In [19]:
X.isna().sum(axis=0)

TestPrep               0
NrSiblings             0
ParentMaritalStatus    0
ReadingScore           0
dtype: int64

In [20]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [21]:
print(str(round(model.intercept_, 3)), end="") # type: ignore
for coeff, col in zip(model.coef_, iksevi):
    print(f" + {str(round(coeff, 3))} \\cdot \\text{{{col}}}", end="")

6.355 + 1.107 \cdot \text{TestPrep} + 0.015 \cdot \text{NrSiblings} + -0.041 \cdot \text{ParentMaritalStatus} + 0.857 \cdot \text{ReadingScore}

$$\text{MathScore} = 6.355 + 1.107 \cdot \text{TestPrep} + 0.015 \cdot \text{NrSiblings} -0.041 \cdot \text{ParentMaritalStatus} + 0.857 \cdot \text{ReadingScore}
$$

## Questions:

### a) Using the summary, write down the formula with coefficients

$$\mathrm{MathScore} = 6.36 + 1.11 \cdot \mathrm{TestPrep} + 0.0154 \cdot  \mathrm{NrSiblings} -0.041 \cdot  \mathrm{ParentMaritalStatus} + 0.857 \cdot  \mathrm{ReadingScore}$$

### b) Are there any statisticaly important features?

### c) Let's interpret the data, if students do test preparation, how does that affect the outcome on the math exam? 

It does help (preparation correlates to higher test scores), but the effect of preparation (1.11 points higher) is not that significant relatively to the total score (100 points).

### d) Are students who practice sport better at math exams or worse?
Using linear regression, model the MathScore using only one feature: PracticeSport. Don't include intercept. Using summary, decide which students do the best at math exams - those who never practice sport, sometimes practice sport or those who do it regularly?

In [33]:
# Declare the model
X = df[["PracticeSport"]]
display(X.describe())
mdl = LinearRegression()
mdl.fit(X, y)

,PracticeSport
count,29117.000000
mean,1.372635
std,0.709069
min,0.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


LinearRegression()

In [34]:
mdl.coef_[0]

0.369943150088119

In [47]:
logistics = LogisticRegression()
y_pred = cross_val_predict(logistics, X, y, cv=2)
print(classification_report(y, y_pred))

c:\Users\Luka\anaconda3\envs\lab1\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Luka\anaconda3\envs\lab1\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Luka\anaconda3\envs\lab1\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    

In [51]:
mdl.summary()

AttributeError: 'LinearRegression' object has no attribute 'summary'